In [1]:
import pandas as pd
import numpy as np
import os
import openpyxl
import psycopg2

In [5]:
df = pd.read_excel("Item.xlsx")

df.head()

,Item ID,Item Name,SKU,Description,Rate,Account,Account Code,Tax Name,Tax Percentage,Tax Type,...,Purchase Account Code,Purchase Description,Inventory Account,Inventory Account Code,Reorder Point,Vendor,Opening Stock,Opening Stock Value,Stock On Hand,Item Type
0,2620362000000082040,Sea Freight Export,NaN,NaN,0.0,Freight,211000012.0,NaN,NaN,NaN,...,221220001.0,NaN,Inventory Asset,113000002.0,NaN,NaN,NaN,NaN,NaN,Sales and Purchases
1,2620362000000082105,Sea Freight Import,NaN,NaN,0.0,Freight,211000012.0,NaN,NaN,NaN,...,221220001.0,NaN,Inventory Asset,113000002.0,NaN,NaN,NaN,NaN,NaN,Sales and Purchases
2,2620362000000082180,Origin Custom Duties,NaN,NaN,0.0,Customs brokerage,211000014.0,NaN,NaN,NaN,...,221221004.0,NaN,Inventory Asset,113000002.0,NaN,NaN,NaN,NaN,NaN,Sales and Purchases
3,2620362000000082291,Air Freight Export,NaN,NaN,0.0,Air,211000002.0,NaN,NaN,NaN,...,221211001.0,NaN,Inventory Asset,113000002.0,NaN,NaN,NaN,NaN,NaN,Sales and Purchases
4,2620362000000082427,Sea Export - 1st / Last mile transportation,NaN,NaN,0.0,First/Last mile,211000017.0,NaN,NaN,NaN,...,221221002.0,NaN,Inventory Asset,113000002.0,NaN,NaN,NaN,NaN,NaN,Sales and Purchases


In [8]:
df.style.format({
    'Purchase Account Code': '{:.0f}'.format
})

,Item ID,Item Name,SKU,Description,Rate,Account,Account Code,Tax Name,Tax Percentage,Tax Type,Purchase Tax Name,Purchase Tax Percentage,Purchase Tax Type,Product Type,Source,Reference ID,Last Sync Time,Status,Usage unit,Purchase Rate,Purchase Account,Purchase Account Code,Purchase Description,Inventory Account,Inventory Account Code,Reorder Point,Vendor,Opening Stock,Opening Stock Value,Stock On Hand,Item Type
0,2620362000000081920,Sea Freight Export,nan,nan,0.000000,Freight,211000012.000000,nan,nan,nan,nan,nan,nan,goods,1,nan,nan,Active,nan,0.000000,Air or Sea Freight,221220001,nan,Inventory Asset,113000002.000000,nan,nan,nan,nan,nan,Sales and Purchases
1,2620362000000081920,Sea Freight Import,nan,nan,0.000000,Freight,211000012.000000,nan,nan,nan,nan,nan,nan,service,1,nan,nan,Active,nan,0.000000,Air or Sea Freight,221220001,nan,Inventory Asset,113000002.000000,nan,nan,nan,nan,nan,Sales and Purchases
2,2620362000000082432,Origin Custom Duties,nan,nan,0.000000,Customs brokerage,211000014.000000,nan,nan,nan,nan,nan,nan,service,1,nan,nan,Active,nan,0.000000,Customs brokerage costs,221221004,nan,Inventory Asset,113000002.000000,nan,nan,nan,nan,nan,Sales and Purchases
3,2620362000000082432,Air Freight Export,nan,nan,0.000000,Air,211000002.000000,nan,nan,nan,nan,nan,nan,service,1,nan,nan,Active,nan,0.000000,Air Freight-Export,221211001,nan,Inventory Asset,113000002.000000,nan,nan,nan,nan,nan,Sales and Purchases
4,2620362000000082432,Sea Export - 1st / Last mile transportation,nan,nan,0.000000,First/Last mile,211000017.000000,nan,nan,nan,nan,nan,nan,service,1,nan,nan,Active,nan,0.000000,First/last Mile delivery,221221002,nan,Inventory Asset,113000002.000000,nan,nan,nan,nan,nan,Sales and Purchases
5,2620362000000082432,Sea Export - 1s / Last mile labour,nan,nan,0.000000,First/Last mile,211000017.000000,nan,nan,nan,nan,nan,nan,service,1,nan,nan,Active,nan,0.000000,First/last Mile delivery,221221002,nan,Inventory Asset,113000002.000000,nan,nan,nan,nan,nan,Sales and Purchases
6,2620362000000082432,Sea Export - Shipping line charges,nan,nan,0.000000,Logistics,211000013.000000,nan,nan,nan,nan,nan,nan,service,1,nan,nan,Active,nan,0.000000,Logistics costs,221221007,nan,Inventory Asset,113000002.000000,nan,nan,nan,nan,nan,Sales and Purchases
7,2620362000000079360,Sea Import Destination Customs Duties,nan,nan,0.000000,Customs brokerage,211000014.000000,nan,nan,nan,nan,nan,nan,service,1,nan,nan,Active,nan,0.000000,Customs brokerage costs,221221004,nan,Inventory Asset,113000002.000000,nan,nan,nan,nan,nan,Sales and Purchases
8,2620362000000079360,Sea Export Terminal Charges,nan,nan,0.000000,Logistics,211000013.000000,nan,nan,nan,nan,nan,nan,service,1,nan,nan,Active,nan,0.000000,Logistics costs,221221007,nan,Inventory Asset,113000002.000000,nan,nan,nan,nan,nan,Sales and Purchases
9,2620362000000079360,Sea Import Terminal Charges,nan,nan,0.000000,Logistics,211000013.000000,nan,nan,nan,nan,nan,nan,service,1,nan,nan,Active,nan,0.000000,Logistics costs,221221007,nan,Inventory Asset,113000002.000000,nan,nan,nan,nan,nan,Sales and Purchases


In [44]:
df.columns=[x.lower().replace(" ","_").replace(".","") for x in df.columns]
df.columns

Index(['item_id', 'item_name', 'sku', 'description', 'rate', 'account',
       'account_code', 'tax_name', 'tax_percentage', 'tax_type',
       'purchase_tax_name', 'purchase_tax_percentage', 'purchase_tax_type',
       'product_type', 'source', 'reference_id', 'last_sync_time', 'status',
       'usage_unit', 'purchase_rate', 'purchase_account',
       'purchase_account_code', 'purchase_description', 'inventory_account',
       'inventory_account_code', 'reorder_point', 'vendor', 'opening_stock',
       'opening_stock_value', 'stock_on_hand', 'item_type'],
      dtype='object')

In [45]:
df.dtypes

item_id                      int64
item_name                   object
sku                        float64
description                float64
rate                       float64
account                     object
account_code               float64
tax_name                   float64
tax_percentage             float64
tax_type                   float64
purchase_tax_name          float64
purchase_tax_percentage    float64
purchase_tax_type          float64
product_type                object
source                       int64
reference_id               float64
last_sync_time             float64
status                      object
usage_unit                 float64
purchase_rate              float64
purchase_account            object
purchase_account_code      float64
purchase_description       float64
inventory_account           object
inventory_account_code     float64
reorder_point              float64
vendor                     float64
opening_stock              float64
opening_stock_value 

In [31]:
replacements = {
    'object' : 'varchar',
    'float64' : 'float',
    'int64' : 'varchar',
    'datetime64[ns]': 'varchar',
    'datetime64': 'timestamp',
    'bool' : 'varchar'
}
replacements

{'object': 'varchar',
 'float64': 'float',
 'int64': 'varchar',
 'datetime64[ns]': 'varchar',
 'datetime64': 'timestamp',
 'bool': 'varchar'}

In [32]:
col_str = ", ".join("{} {}".format(n, d) for (n, d) in zip(df.columns, df.dtypes.replace(replacements)))
col_str

'item_id varchar, item_name varchar, sku float, description float, rate float, account varchar, account_code float, tax_name float, tax_percentage float, tax_type float, purchase_tax_name float, purchase_tax_percentage float, purchase_tax_type float, product_type varchar, source varchar, reference_id float, last_sync_time float, status varchar, usage_unit float, purchase_rate float, purchase_account varchar, purchase_account_code float, purchase_description float, inventory_account varchar, inventory_account_code float, reorder_point float, vendor float, opening_stock float, opening_stock_value float, stock_on_hand float, item_type varchar'

In [33]:
conn_string = "host= db.app.production.jetstreamafrica.com \
                dbname='jetstream-finance' \
                    user='ujetstreamfinance' password='Smehr5A7FGw4jEX@' "

conn = psycopg2.connect(conn_string)
cursor = conn.cursor()

print('opened database successfully')

opened database successfully


In [34]:
cursor.execute("drop table if exists items;")

In [35]:
cursor.execute("create table items \
    (item_id varchar, item_name varchar, sku float, description varchar, rate varchar, account varchar, account_code varchar, tax_name varchar, tax_percentage float, tax_type float, purchase_tax_name float, purchase_tax_percentage float, purchase_tax_type float, product_type varchar, source varchar, reference_id varchar, last_sync_time varchar, status varchar, usage_unit float, purchase_rate varchar, purchase_account varchar, purchase_account_code float, purchase_description varchar, inventory_account varchar, inventory_account_code varchar, reorder_point float, vendor varchar, opening_stock float, opening_stock_value float, stock_on_hand float, item_type varchar)")

In [36]:
#insert values to tables
#save df to csv
df.to_csv('items', header=df.columns, index=False, encoding='utf-8')
#open the csv file, save it as an object and upload to db
my_file = open("items")
print('file opened in memory')

file opened in memory


In [37]:
SQL_STATEMENT = """
COPY items FROM STDIN WITH
   CSV
   HEADER
   DELIMITER AS ','
"""
cursor.copy_expert(sql = SQL_STATEMENT, file=my_file)
print('file copied to db')

file copied to db


In [38]:
cursor.execute("grant select on table items to public")
conn.commit()

cursor.close()
print('table items imported successfully')

table items imported successfully
